# Modelagem

In [1]:
import time
import logging

import numpy as np
import pandas as pd

from pathlib import Path
from typing import (
    Callable,
    TypeVar,
    Sequence,
    TypeAlias,
)
import random

In [2]:
RNG: TypeAlias = np.random.Generator
T = TypeVar("T")

FitnessFunc: TypeAlias = Callable[[T], float]
PairingFunc: TypeAlias = Callable[[Sequence[T], Sequence[float]], Sequence[tuple[T, T]]]
SurvivorFunc: TypeAlias = Callable[[RNG, Sequence[T], Sequence[float], int], list[T]]
CrossoverFunc: TypeAlias = Callable[[RNG, T, T], T]
SelectionFunc: TypeAlias = Callable[
    [RNG, Sequence[T], Sequence[float], int], Sequence[T]
]
MutationFunc: TypeAlias = Callable[[RNG, T], T]
PopulationFactory: TypeAlias = Callable[[RNG, int], list[T]]

In [3]:

OUTPUT_PATH = Path("./out").resolve().absolute()
ENABLE_LOGGING = True

if ENABLE_LOGGING:
    level = logging.INFO
    fmt = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    date_fmt = "%Y-%m-%d %H:%M:%S"
    log_filter = logging.Filter("ES-VND")

    file_handler = logging.FileHandler(OUTPUT_PATH / "log.txt", delay=True, mode="a")
    file_handler.setLevel(level)
    file_handler.setFormatter(logging.Formatter(fmt, datefmt=date_fmt))
    file_handler.addFilter(log_filter)

    console_handler = logging.StreamHandler()
    console_handler.setLevel(level)
    console_handler.setFormatter(logging.Formatter(fmt, datefmt=date_fmt))
    console_handler.set_name(logging.getLevelName(level))
    file_handler.addFilter(log_filter)
    logging.basicConfig(
        level=level,
        format=fmt,
    )

    logging.basicConfig(
        level=level,
        format=fmt,
        datefmt=date_fmt,
        force=True,
        handlers=[
            file_handler,
            console_handler,
        ],
    )

In [4]:
# IntArray = np.ndarray[tuple[int], np.dtype[np.int64]]

# Fitness

In [5]:
def qap_fitness(solution: list[int], flow: np.ndarray, dist: np.ndarray) -> float:
    n = len(solution)
    total_cost = 0
    for i in range(n):
        for j in range(n):
            total_cost += flow[i][j] * dist[solution[i]][solution[j]]
    return total_cost

In [6]:
def calcular_aptidao(individuo):
    return sum(individuo)

In [7]:
class Solucao:
    _size: int
    _genes: np.ndarray

    def __init__(self, size: int):
        self._size = size
        self._genes = np.random.permutation(size)

    def state(self):
        return self._genes.copy()

    def size(self):
        return self._size

    def __repr__(self):
        return f"Solucao(size={self.size()}, genes={self.state()})"

# Mutação

In [8]:
def mutacao(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

In [9]:
def gerar_vizinho(solucao):
    return mutacao(solucao)

In [10]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    random.seed(seed)
    vetor = list(range(tamanho))  
    random.shuffle(vetor)
    return vetor

In [11]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    n = len(solucao)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao[i]][solucao[j]]
    return custo

In [12]:
def VND(solucao, matriz_fluxo, matriz_distancia, kmax=5):
    melhor = solucao[:]
    k = 1
    while k <= kmax:
        vizinho = gerar_vizinho(melhor)
        if calcular_aptidao_qap(vizinho, matriz_fluxo, matriz_distancia) < calcular_aptidao_qap(melhor, matriz_fluxo, matriz_distancia):
            melhor = vizinho
            k = 1
        else:
            k += 1
    return melhor

In [13]:
def ES_VND(mu, lambd, tempo_max, taxa_mutacao, taxa_busca_local, iter_sem_melhora_max,
           solucao_inicial, matriz_fluxo, matriz_distancia):

    P = [mutacao(solucao_inicial) for _ in range(lambd)]
    melhor = None
    melhor_aptidao = float('inf')
    sem_melhora = 0

    inicio = time.time()

    while (time.time() - inicio) < tempo_max and sem_melhora < iter_sem_melhora_max:
        aptidoes = [calcular_aptidao_qap(ind, matriz_fluxo, matriz_distancia) for ind in P]

        for ind, apt in zip(P, aptidoes):
            if apt < melhor_aptidao:
                melhor_aptidao = apt
                melhor = ind[:]
                sem_melhora = 0
        sem_melhora += 1

        melhores_indices = sorted(range(len(P)), key=lambda i: aptidoes[i])[:mu]
        Q = [P[i][:] for i in melhores_indices]

        nova_geracao = Q[:]
        for q in Q:
            for _ in range(lambd // mu):
                individuo = q[:]
                if random.random() < taxa_mutacao:
                    individuo = mutacao(individuo)
                if random.random() < taxa_busca_local:
                    individuo = VND(individuo, matriz_fluxo, matriz_distancia)
                nova_geracao.append(individuo)

        P = nova_geracao

    return melhor, melhor_aptidao


In [14]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [15]:
if __name__ == "__main__":
    n, flow_df, dist_df = ler_qap_com_n("Els19.txt")
    flow = flow_df.values.tolist()
    dist = dist_df.values.tolist()
    matriz_fluxo = np.array(flow)
    matriz_distancia = np.array(dist)
    solucao_inicial = gerar_array_replicavel(seed=42, tamanho=n)

    parametros = {
        "mu": random.randint(5, 10),
        "lambd": random.randint(30, 100),
        "tempo_max": random.randint(3, 10) * 60,
        "taxa_mutacao": random.uniform(0.4, 0.7),
        "taxa_busca_local": random.uniform(0.4, 0.7),
        "iter_sem_melhora_max": random.randint(5, 10),
        "solucao_inicial": solucao_inicial,
        "matriz_fluxo": matriz_fluxo,
        "matriz_distancia": matriz_distancia
    }

    melhor_solucao, melhor_valor = ES_VND(**parametros)
    print("Melhor solução encontrada:", melhor_solucao)
    print("Custo:", melhor_valor)
    print("Parâmetros usados:", parametros)

Melhor solução encontrada: [14, 15, 7, 11, 10, 17, 12, 16, 5, 8, 9, 4, 6, 3, 13, 18, 0, 1, 2]
Custo: 17997928
Parâmetros usados: {'mu': 9, 'lambd': 55, 'tempo_max': 540, 'taxa_mutacao': 0.466132186612209, 'taxa_busca_local': 0.5767797051627725, 'iter_sem_melhora_max': 5, 'solucao_inicial': [14, 13, 4, 9, 5, 15, 17, 6, 12, 16, 10, 1, 11, 2, 18, 7, 8, 0, 3], 'matriz_fluxo': array([[  0,  12,  36,  28,  52,  44, 110, 126,  94,  63, 130, 102,  65,
         98, 132, 132, 126, 120, 126],
       [ 12,   0,  24,  75,  82,  75, 108,  70, 124,  86,  93, 106,  58,
        124, 161, 161,  70,  64,  70],
       [ 36,  24,   0,  47,  71,  47, 110,  73, 126,  71,  95, 110,  46,
        127, 163, 163,  73,  67,  73],
       [ 28,  75,  47,   0,  42,  34, 148, 111, 160,  52,  94, 148,  49,
        117, 104, 109, 111, 105, 111],
       [ 52,  82,  71,  42,   0,  42, 125, 136, 102,  22,  73, 125,  32,
         94, 130, 130, 136, 130, 136],
       [ 44,  75,  47,  34,  42,   0, 148, 111, 162,  52,  96, 14